# Initialise Python env and helper functions

## Configure the environement

In [1]:
# Install packages quietly using the -q flag to suppress output
!pip install -q latexify-py > /dev/null 2>&1
!pip install -q openai==1.58.1 langchain-core langchain-openai > /dev/null 2>&1

# Create a requirements.txt file with only our packages and their versions.
# This command filters the pip freeze output to include just the packages of interest.
!pip freeze | grep -E '^(latexify-py|openai|langchain-core|langchain-openai)==' > requirements.txt

# Display the current Python version
!python --version

Python 3.10.12


In [2]:
import math
import latexify
from IPython.display import display, Math, Latex
from sympy import *

In [3]:
import os
import datetime
import json
import traceback

## LLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

from IPython.display import display, Markdown

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
OPENAI_API_KEY = user_secrets.get_secret("openai_key")

In [4]:
BASE_LLM = 'gpt-4o-2024-05-13'
ADVANCED_LLM = 'o3-mini'
SELECTED_LLM = ADVANCED_LLM
TEMPERATURE = 0
MAX_TOKENS=3000

## Helper function to solve math with LLM

In [5]:
def solve_math_problem(context, file_path='/kaggle/working/problem_solution.md'):
    """
    Solve a math problem by constructing a prompt, invoking the LLM chain,
    and saving the result to a Markdown file with a timestamp in its name.

    Parameters:
        context (str): The problem statement or context.
        file_path (str): The base path where the Markdown report will be saved.
                         The timestamp will be appended to this filename.
                         Defaults to '/kaggle/working/problem_solution.md'.

    Returns:
        result (str): The generated answer in markdown format, or None if an error occurred.
    """
    # Append a timestamp to the file name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Split the file_path into the base and extension (assumes file_path has an extension)
    base, ext = file_path.rsplit('.', 1)
    file_path_with_timestamp = f"{base}_{timestamp}.{ext}"

    # Define a default template with a placeholder for the context
    template = """
Solve the problem below. Explain your reasoning step by step. Return the result in markdown format.
When returning equations in the Latex format make sure to use $$ $$ around them to ensure they render in markdown.
{context}
    """
    
    # Create a ChatPromptTemplate from the template
    prompt = ChatPromptTemplate.from_template(template)
    
    # Prepare parameters for ChatOpenAI
    model_params = {
        "model": SELECTED_LLM,
        "api_key": OPENAI_API_KEY
    }
    
    display(Markdown(f'**Selected model: {SELECTED_LLM}**'))
    
    # Conditionally set temperature and max_tokens if the model supports these parameters
    if SELECTED_LLM != ADVANCED_LLM:
        model_params["temperature"] = TEMPERATURE 
        model_params["max_tokens"] = MAX_TOKENS
    
    # Initialize the ChatOpenAI model with the specified parameters
    model = ChatOpenAI(**model_params)
    
    # Create the processing chain by composing the prompt, model, and output parser
    chain = prompt | model | StrOutputParser()
    
    try:
        # Invoke the chain to generate the answer based on the context
        result = chain.invoke(context)
    
        # Save both the prompt and the result to a Markdown file with a timestamp in its name
        with open(file_path_with_timestamp, 'w') as f:
            f.write("# Question\n\n")
            f.write("## Prompt\n")
            f.write(template.format(context=context))
            f.write("\n\n## Answer\n")
            f.write(result)
    
        # Display the result and the file path in the notebook
        display(Markdown(result))
        display(Markdown(f"**Markdown report saved to: {file_path_with_timestamp}**"))
    
        return result
    
    except BadRequestError as e:
        print(f"An error occurred: {e}")
        return None

In [6]:
import datetime
from IPython.display import Markdown, display

def solve_math_problem(context, template=None, file_path='/kaggle/working/problem_solution.md'):
    """
    Solve a math problem by constructing a prompt, invoking the LLM chain,
    and saving the result to a Markdown file with a timestamp in its name.

    Parameters:
        context (str): The problem statement or context.
        template (str): A prompt template with a placeholder for the context.
                        If None, a default template is used.
        file_path (str): The base path where the Markdown report will be saved.
                         A timestamp is appended to this filename.
                         Defaults to '/kaggle/working/problem_solution.md'.

    Returns:
        result (str): The generated answer in markdown format, or None if an error occurred.
    """
    # Append a timestamp to the file name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    # Split the file_path into the base and extension (assumes file_path has an extension)
    base, ext = file_path.rsplit('.', 1)
    file_path_with_timestamp = f"{base}_{timestamp}.{ext}"

    # Use the provided template or fall back to the default one
    if template is None:
        template = """
Solve the problem below. Explain your reasoning step by step. Return the result in markdown format.
When returning equations in the Latex format make sure to use $$ $$ around them to ensure they render in markdown.
{context}
        """
    
    # Create a ChatPromptTemplate from the template
    prompt = ChatPromptTemplate.from_template(template)
    
    # Prepare parameters for ChatOpenAI
    model_params = {
        "model": SELECTED_LLM,
        "api_key": OPENAI_API_KEY
    }
    
    display(Markdown(f'**Selected model: {SELECTED_LLM}**'))
    
    # Conditionally set temperature and max_tokens if the model supports these parameters
    if SELECTED_LLM != ADVANCED_LLM:
        model_params["temperature"] = TEMPERATURE 
        model_params["max_tokens"] = MAX_TOKENS
    
    # Initialize the ChatOpenAI model with the specified parameters
    model = ChatOpenAI(**model_params)
    
    # Create the processing chain by composing the prompt, model, and output parser
    chain = prompt | model | StrOutputParser()
    
    try:
        # Invoke the chain to generate the answer based on the context
        result = chain.invoke(context)
    
        # Save both the prompt and the result to a Markdown file with a timestamp in its name
        with open(file_path_with_timestamp, 'w') as f:
            f.write("# Question\n\n")
            f.write("## Prompt\n")
            f.write(template.format(context=context))
            f.write("\n\n## Answer\n")
            f.write(result)
    
        # Display the result and the file path in the notebook
        display(Markdown(result))
        display(Markdown(f"**Markdown report saved to: {file_path_with_timestamp}**"))
    
        return result
    
    except BadRequestError as e:
        print(f"An error occurred: {e}")
        return None

# Permutations

## Q1

- Permutations: Evaluate P(5,3)

In [7]:
SELECTED_LLM = ADVANCED_LLM

my_template = """
Solve the problem below. Explain your reasonning step by step. Return the result in markdown format. When possible also write a Python script to solve the problem.
Put the python code in a ```python ``` block. 
When returning equations in the Latex format make sure to use $$ $$ around them to make sure they will render in the markdown format.  
{context}
"""

# Define your problem context
problem_context = """
Permutations: Evaluate P(5,3)
"""

# Solve the problem and get the result
result = solve_math_problem(problem_context, my_template)

**Selected model: o3-mini**

We are asked to compute $$P(5,3)$$, which represents the number of ways to arrange 3 objects selected from 5 distinct objects.

### Step-by-step Reasoning

1. **Definition**: The permutation formula for selecting and arranging \( k \) items from \( n \) items is given by:
   $$ 
   P(n, k) = \frac{n!}{(n-k)!} 
   $$

2. **Substitute the Values**: For \( n=5 \) and \( k=3 \), substitute into the formula:
   $$ 
   P(5, 3) = \frac{5!}{(5-3)!} = \frac{5!}{2!} 
   $$

3. **Evaluate the Factorials**:
   - \( 5! = 5 \times 4 \times 3 \times 2 \times 1 = 120 \)
   - \( 2! = 2 \times 1 = 2 \)

4. **Compute the Final Value**:
   $$ 
   P(5, 3) = \frac{120}{2} = 60 
   $$

Therefore, there are **60** ways to arrange 3 items from a set of 5.

---

Here is a Python script that computes this result:

```python
import math

# Calculate P(5,3) using the permutation formula
result = math.factorial(5) // math.factorial(5 - 3)
print("P(5,3) =", result)
```

Alternatively, if you're using Python 3.8 or higher, you can use the `math.perm` function:

```python
import math

result = math.perm(5, 3)
print("P(5,3) =", result)
```

Both scripts will output:

```
P(5,3) = 60
```

Thus, the evaluated result of \( P(5,3) \) is **60**.

**Markdown report saved to: /kaggle/working/problem_solution_20250208_141720.md**

## Q2

- compute: $(^{}_6P{}_1 =)$

In [8]:
SELECTED_LLM = BASE_LLM

my_template = """
Solve the problem below. Explain your reasonning step by step. Return the result in markdown format. When possible also write a Python script to solve the problem.
Put the python code in a ```python ``` block. 
When returning equations in the Latex format make sure to use $$ $$ around them to make sure they will render in the markdown format.  
{context}
"""

# Define your problem context
problem_context = """
compute: $(^{}_6P{}_1 =)$
"""

# Solve the problem and get the result
result = solve_math_problem(problem_context, my_template)

**Selected model: gpt-4o-2024-05-13**

To solve the problem of computing the permutation \( P(6, 1) \), we need to understand the formula for permutations. The permutation formula is given by:

$$
P(n, k) = \frac{n!}{(n - k)!}
$$

Where:
- \( n \) is the total number of items,
- \( k \) is the number of items to choose,
- \( n! \) denotes the factorial of \( n \).

In this case, we have \( n = 6 \) and \( k = 1 \). Plugging these values into the formula, we get:

$$
P(6, 1) = \frac{6!}{(6 - 1)!} = \frac{6!}{5!}
$$

We know that \( 6! = 6 \times 5 \times 4 \times 3 \times 2 \times 1 \) and \( 5! = 5 \times 4 \times 3 \times 2 \times 1 \). Therefore:

$$
P(6, 1) = \frac{6 \times 5 \times 4 \times 3 \times 2 \times 1}{5 \times 4 \times 3 \times 2 \times 1} = \frac{6}{1} = 6
$$

So, the result of \( P(6, 1) \) is 6.

Here is the Python code to compute this:

```python
import math

def permutation(n, k):
    return math.factorial(n) // math.factorial(n - k)

result = permutation(6, 1)
print(result)
```

This code uses the `math.factorial` function to compute the factorials and then calculates the permutation using integer division. The result will be printed as 6.

**Markdown report saved to: /kaggle/working/problem_solution_20250208_141725.md**

## Q3

- Of the 7 students who are going to take a test, the top 2 will get different prizes. In how many ways can both prizes be awarded?

In [9]:
SELECTED_LLM = BASE_LLM

# Define your problem context
problem_context = """
Of the 7 students who are going to take a test, the top 2 will get different prizes. In how many ways can both prizes be awarded?
"""

# Solve the problem and get the result
result = solve_math_problem(problem_context, my_template)

**Selected model: gpt-4o-2024-05-13**

To solve the problem of determining the number of ways to award the top 2 prizes to 7 students, we need to consider the following:

1. **Selection of the top 2 students**: We need to choose 2 students out of 7. The order in which we choose these students matters because the prizes are different.

2. **Permutations**: Since the order matters, we are dealing with permutations, not combinations.

### Step-by-Step Reasoning

1. **First Prize**: We have 7 choices for the first prize.
2. **Second Prize**: After awarding the first prize, we have 6 remaining choices for the second prize.

Thus, the total number of ways to award the two different prizes is given by the product of the number of choices for each prize.

### Calculation

The number of ways to award the prizes is:
$$
7 \times 6 = 42
$$

So, there are 42 different ways to award the top 2 prizes to 7 students.

### Python Script

Here is a Python script to calculate the number of ways to award the prizes:

```python
def calculate_ways_to_award_prizes(total_students, prizes):
    ways = 1
    for i in range(prizes):
        ways *= (total_students - i)
    return ways

total_students = 7
prizes = 2
ways_to_award_prizes = calculate_ways_to_award_prizes(total_students, prizes)
print(f"The number of ways to award the top 2 prizes to {total_students} students is: {ways_to_award_prizes}")
```

This script defines a function `calculate_ways_to_award_prizes` that calculates the number of ways to award the prizes by multiplying the number of choices for each prize. It then prints the result.

### Result

The number of ways to award the top 2 prizes to 7 students is:
$$
42
$$

**Markdown report saved to: /kaggle/working/problem_solution_20250208_141730.md**